In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
import glove
from glove import Corpus
import time

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, Concatenate

from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
# from keras.backend.tensorflow_backend import set_session, clear_session, get_session
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')
def mean(a):
    return sum(a) / len(a)

2023-04-23 20:35:20.580782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 20:35:20.645867: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 20:35:20.646760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 20:35:21.569826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = "results/timeseries-baseline/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [3]:
def timeseries_model(layer_name, number_of_unit):
    K.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [1]:
timer_file_path = "/home/ubuntu/notebook/time_results/07/"

model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

epoch_num = 100
unit_sizes = [128, 256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["LSTM", "GRU"]


# epoch_num = 2
# iter_num = 2
# target_problems = ['mort_hosp']
# layers = ["GRU"]
# unit_sizes = [256]

time_saver = dict()

for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for cycle in range(0, iter_num):
            iteration = cycle + 1
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")

                initial_time = time.time()

                # early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                # best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                # checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                #     save_best_only=True, mode='min', save_freq='epoch')

                # callbacks = [early_stopping_monitor, checkpoint]

                # model = timeseries_model(each_layer, each_unit_size)
                # model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1, 
                #           validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                # model.load_weights(best_model_name)

                # probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                # save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                #                        each_problem, iteration, each_unit_size,type_of_ner)
                # reset_keras(model)
                # #del model
                # clear_session()
                # gc.collect()

                final_time = time.time() - initial_time
                with open(f"{timer_file_path}{each_layer}_{each_unit_size}_{iteration}_{each_problem}") as timer_file: 
                    if each_layer not in time_saver: 
                        time_saver[each_layer] = dict()
                    if each_unit_size not in time_saver[each_layer]:
                        time_saver[each_layer][each_unit_size] = dict()
                    if each_problem not in time_saver[each_layer][each_unit_size]: 
                        time_saver[each_layer][each_unit_size][each_problem] = list()
                    
                    time_saver[each_layer][each_unit_size][each_problem].append(final_time)
                    timer_file.write(final_time)

SyntaxError: EOL while scanning string literal (<ipython-input-1-26239e90c478>, line 56)

In [ ]:
for layer in time_saver:
    for unit in time_saver[layer]: 
        for problem in time_saver[layer][unit]:
            print(time_saver[layer][unit][problem])